In [ ]:
import isofate as iso
import pickle
import time as TIME

import matplotlib.pyplot as plt

import isofate.constants as const

from isofate.isofate_coupler import isocalc

from isofate.isofunks import *
from isofate.orbit_params import SemiMajor, Insolation, EqTemp
from isofate.isoplot import isoplot


# Setting model parameters

### Define stellar parameters

In [ ]:
# # LHS 1140
R_star = 0.22*const.Rs # [m]
M_star = 0.18*const.Ms # [kg]
T_star = 3096 # [K]
t_jump = 5.9 - 15.4*(M_star/const.Ms)
L = 0.0038*const.Ls
stellar_type = 'M1'

### Define planet parameters

In [ ]:
f_atm = 0.02
P = 5/const.s2day
a = SemiMajor(M_star, P) # [m]
d = a

Mp = 5*const.Me
M_atm = Mp*f_atm # initial atmospheric mass [kg]

### Set Stellar Flux properties 

In [ ]:
Fp = Insolation(L, a)  # [W/m2]
T = EqTemp(Fp, A = 0) # planetary eq temp [K]

F0 = Fp*1e-3 # use for M star 
F_final = 0.170 # 0.170 for GJ 699 MUSCLES; use 0.033 for GJ 1132 MUSCLES
flux_model = 'power law'

### Set simulation times

In [ ]:
t_sat = 5e8 # XUV saturation time [yr]
# t_sat = t_jump*1e9 # XUV saturation time [yr]
d = a # orbital distance [m]
time = 5e9 # total simulation time [yr]
t0 = 1e6 # start time [yr]
t_pms = 0 # pms phase duration [yr]

step_fn = False

### Atmodeller parameters

In [ ]:
mechanism = 'XUV' # if using fixed phi, be sure to change Rp = r_core below and rad_evol = False
RR = True
rad_evol = True
Rp_override = False
n_steps = int(1e5)
n_atmodeller = int(1e4)
thermal = True

melt_fraction_override = False
save_molecules = True
# mantle_iron_dict = {'type': 'static', 'Fe_mass_fraction': 0.1}
mantle_iron_dict = False

OtoH_enhancement = 1
OtoH_enhanced = const.OtoH_protosolar*OtoH_enhancement
OtoH_enhanced_mass = OtoH_enhanced*(const.mu_O/const.mu_H)

### Setting number of species in atmosphere

In [ ]:
N_He = (const.HetoH_protosolar_mass/(1 + const.HetoH_protosolar_mass))*M_atm/const.mu_He # initial He number [atoms]# N_He = 0
N_H = (1 - const.DtoH_solar_mass - OtoH_enhanced_mass - const.CtoH_protosolar_mass - const.NtoH_protosolar_mass - const.StoH_protosolar_mass)*M_atm/(1 + const.HetoH_protosolar_mass)/const.mu_H  # initial H number [atoms]
N_D = const.DtoH_solar_mass*M_atm/(1 + const.HetoH_protosolar_mass)/const.mu_D
N_O = OtoH_enhanced_mass*M_atm/(1 + const.HetoH_protosolar_mass)/const.mu_O
N_C = const.CtoH_protosolar_mass*M_atm/(1 + const.HetoH_protosolar_mass)/const.mu_C
mu_avg = (N_H*const.mu_H + N_He*const.mu_He + N_D*const.mu_D + N_O*const.mu_O + N_C*const.mu_C)/(N_H+N_He+N_D+N_O+N_C)

# Run IsoFATE

In [ ]:
sol = isocalc(f_atm, Mp, M_star, F0, Fp, T, d, time, mechanism, rad_evol,
              N_H = N_H, N_He = N_He, N_D = N_D, N_O = N_O, N_C = N_C, melt_fraction_override = melt_fraction_override,
              mu = mu_avg, eps = 0.15, activity = 'medium', flux_model = flux_model, stellar_type = stellar_type, 
              Rp_override = Rp_override, t_sat = t_sat, step_fn = step_fn, F_final = F_final, t_pms = t_pms, pms_factor = 1e2,
              n_steps = n_steps, t0 = t0, rho_rcb = 1.0, RR = RR,
              thermal = thermal, beta = -1.23, n_atmodeller = n_atmodeller, 
              save_molecules = save_molecules, mantle_iron_dict = mantle_iron_dict)

### Plot Outputs

In [ ]:
isoplot(sol, n_atmodeller, Mp, f_atm, Fp, T, M_star, d)